### โค้ดไพทอนประกอบบทที่ 4 ของหนังสือ 
### "การโปรแกรมไพทอนสำหรับงานควบคุมและฝังตัว"
### ดร.วโรดม ตู้จินดา

<b>หมายเหตุ : </b> โค้ดบางส่วนใน notebook นี้ที่ใช้ tkinter ไม่สามารถรันบน colab

<a href="https://colab.research.google.com/github/dewdotninja/py4conemb/blob/master/code/chapter8/notebooks/chapter8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## หากรันใน colab หรือยังไม่มีไลบรารี Python control library  ต้องติดตั้งก่อนโดยรันเซลล์นี้
!pip install control

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import control as ctl

In [ ]:
#Plan is to make 2-link controller an IoT. Then implement inverse kinematics 
#and plot 2-link manipulator here. Make robot changes its pose when new joint variables are received.


In [ ]:
import paho.mqtt.client as mqtt

In [ ]:
Client_ID = ""
User = ""
Password = ""
client = mqtt.Client(client_id=Client_ID,
                         transport='tcp',
                         protocol=mqtt.MQTTv311,
                         clean_session=True)
client.username_pw_set(User,Password)

In [ ]:
broker = "broker.netpie.io"
myport = 1883
client.connect(broker, port=myport, keepalive=60)

In [ ]:
client.subscribe("@msg/update")

In [ ]:
def on_message(client, userdata, message):
    global rcvd_msg, rcvd_topic
    rcvd_msg = str(message.payload.decode("utf-8"))
    rcvd_topic = message.topic
    print("message received ", rcvd_msg)
    print("message topic=",message.topic)

In [ ]:
client.on_message=on_message

In [ ]:
client.loop_start()

In [ ]:
vrstat = 1
cmd_str = "vrcmd="+str(vrstat)
client.publish("@msg/cmd",cmd_str)

In [ ]:
client.loop_stop()

In [ ]:
client.disconnect()

In [ ]:
# capture message example
# message received  0.005,1,200,1,0,0.0,0.0,0.0,0.0,0.0,0.0,-0.01324777,-5.526962,93.48776,0.0633755,10.36197,-420.5724,587.4396,-1148.583
# message topic= @msg/update

In [2]:
#!pip install paho-mqtt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=62118 sha256=6ef527dd81825964c2684df20f0acd36c2f1a268131d55ca3370e7c3e8d54909
  Stored in directory: /Users/dew/Library/Caches/pip/wheels/0f/90/29/db29bb8ddc98ec5f2363b959130c9ddbcf5cfdb4a00b6184dd
Successfully built paho-mqtt


In [3]:
# class implementation of robo2links controller GUI
import numpy as np
import paho.mqtt.client as mqtt
import tkinter as tk
from tkinter import *
from tkinter import ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle 
from matplotlib.animation import FuncAnimation

In [ ]:
# Ref: For the following class def, I use code from this link
# https://stackoverflow.com/questions/60413174/rotating-rectangles-around-point-with-matplotlib

In [2]:
class RotatingRectangle(Rectangle):
    def __init__(self, xy, width, height, rel_point_of_rot, **kwargs):
        super().__init__(xy, width, height, **kwargs)
        self.rel_point_of_rot = rel_point_of_rot
        self.xy_center = self.get_xy()
        self.set_angle(self.angle)

    def _apply_rotation(self):
        angle_rad = self.angle * np.pi / 180
        m_trans = np.array([[np.cos(angle_rad), -np.sin(angle_rad)],
                            [np.sin(angle_rad), np.cos(angle_rad)]])
        shift = -m_trans @ self.rel_point_of_rot
        self.set_xy(self.xy_center + shift)

    def set_angle(self, angle):
        self.angle = angle
        self._apply_rotation()

    def set_rel_point_of_rot(self, rel_point_of_rot):
        self.rel_point_of_rot = rel_point_of_rot
        self._apply_rotation()

    def set_xy_center(self, xy):
        self.xy_center = xy
        self._apply_rotation()

In [ ]:

class r2Controller:

    def __init__(self, root):
        
        screen_width = 800
        screen_height = 800
        self.TFont1 = ("Consolas",14)
        self.TFont2 = ("Consolas",16)
        
        self.online = True  # change to True to connect to NETPIE
        self.rad2deg = 180/np.pi
        
        root.title("Robo2links Controller")
        root.protocol("WM_DELETE_WINDOW", self.on_close)
        root.geometry(str(screen_width)+'x'+str(screen_height))
        mainframe = ttk.Frame(root, padding="3 3 12 12")
        mainframe.grid(column=0, row=0, sticky=(N, W, E, S))
        root.columnconfigure(0, weight=1)
        root.rowconfigure(0, weight=1)
        cvrowspan = 15  # canvas row and column span. Increase this from 6 to 14
        cvcolspan = 8  # increase from 3 to 7
        # rectangle size of robot link
        self.link_height = 0.2
        self.link_width = 1        
        # string parameters corresponding to real parmeter values
        self.t_txt = StringVar()
        self.plantsim_txt = StringVar()
        self.datasize_txt = StringVar()
        self.cmdgen_txt = StringVar()
        self.feedback_txt = StringVar()
        self.controller_txt = StringVar()
        
        self.iktype_txt = StringVar()  # select ikine method
        self.ikmsg_txt = StringVar()  # message to alert if no solution
        self.ikmsg_txt.set("OK")  
        self.xg_txt = StringVar()   # goal XY coordinate 
        self.yg_txt = StringVar()
        self.xt_txt = StringVar()  # XY coordinate of tool tip
        self.yt_txt = StringVar()
        
        # joint variables, command and actual
        self.q1r_deg_txt = StringVar()
        self.q1_deg_txt = StringVar()
        self.q2r_deg_txt = StringVar()
        self.q2_deg_txt = StringVar()
        self.q1r_txt = StringVar()
        self.qd1r_txt = StringVar()
        self.qdd1r_txt = StringVar()
        self.q2r_txt = StringVar()
        self.qd2r_txt = StringVar()
        self.qdd2r_txt = StringVar()
        self.q1_txt = StringVar()
        self.qd1_txt = StringVar()
        self.qdd1_txt = StringVar()
        self.q2_txt = StringVar()
        self.qd2_txt = StringVar()
        self.qdd2_txt = StringVar()
        self.tau1_txt = StringVar()
        self.tau2_txt = StringVar()
        
        # update parameters to current value
        # ttk.Button(mainframe, text="Update", command=self.dbupdate).grid(column=0, row=cvrowspan+6, sticky=W)
        # command box 
        #  ttk.Label(mainframe, text="Cmd : ", font=self.TFont1).grid(column=0, row=cvrowspan+5, sticky=E)
        self.r2_txt = StringVar()
        r2cmd_entry = ttk.Entry(mainframe, textvariable=self.r2_txt)
        r2cmd_entry.grid(column=0, row=cvrowspan+5,sticky=(W, E))
        ttk.Button(mainframe, text="Send", command=self.send_r2).grid(column=1, row=cvrowspan+5, sticky=W)

        # joint variable display 
        r2frame = ttk.Frame(mainframe, padding = (2,2,2,2),relief="sunken")
        r2frame.grid(column=0,row=cvrowspan+1)

        # joint variables diaplay --- ref. v.s. actual)
        ttk.Label(r2frame, text="angle (deg)       ", 
                  font=self.TFont1).grid(column=1, row=1, sticky=W)        
        ttk.Label(r2frame, text="angle (rad)       ", 
                  font=self.TFont1).grid(column=2, row=1, sticky=W)
        ttk.Label(r2frame, text="velocity (rad/s)    ", 
                  font=self.TFont1).grid(column=3, row=1, sticky=W)
        ttk.Label(r2frame, text="acceleration (rad/s^2)", 
                  font=self.TFont1).grid(column=4, row=1, sticky=W)
        
        ttk.Label(r2frame, text="J1 (ref.): ",font=self.TFont1).grid(column=0, row=2, sticky=W)
        ttk.Label(r2frame, text="J1 (robot) : ",font=self.TFont1).grid(column=0, row=3, sticky=W)
        ttk.Label(r2frame, text="J2 (ref.) : ",font=self.TFont1).grid(column=0, row=4, sticky=W)
        ttk.Label(r2frame, text="J2 (robot) : ",font=self.TFont1).grid(column=0, row=5, sticky=W)

        # joint variable data from simulation
        #  joint angle (deg) -- ref. v.s. actual
        ttk.Label(r2frame, font=self.TFont1, foreground='blue',
                   textvariable=self.q1r_deg_txt).grid(column=1, row=2, sticky=(W)) 
        ttk.Label(r2frame, font=self.TFont1, foreground='red',
                   textvariable=self.q1_deg_txt).grid(column=1, row=3, sticky=(W)) 
        ttk.Label(r2frame, font=self.TFont1, foreground='blue',
                   textvariable=self.q2r_deg_txt).grid(column=1, row=4, sticky=(W)) 
        ttk.Label(r2frame, font=self.TFont1, foreground='red',
                   textvariable=self.q2_deg_txt).grid(column=1, row=5, sticky=(W)) 
        
        
        #  joint angle (rad) -- ref. v.s. actual
        ttk.Label(r2frame, font=self.TFont1, foreground='brown',
                   textvariable=self.q1r_txt).grid(column=2, row=2, sticky=(W)) 
        ttk.Label(r2frame, font=self.TFont1, foreground='red',
                   textvariable=self.q1_txt).grid(column=2, row=3, sticky=(W)) 
        ttk.Label(r2frame, font=self.TFont1, foreground='brown',
                   textvariable=self.q2r_txt).grid(column=2, row=4, sticky=(W)) 
        ttk.Label(r2frame, font=self.TFont1, foreground='red',
                   textvariable=self.q2_txt).grid(column=2, row=5, sticky=(W)) 
        #  joint velocity -- ref. v.s. actual
        ttk.Label(r2frame, font=self.TFont1, foreground='green',
                   textvariable=self.qd1r_txt).grid(column=3, row=2, sticky=(W)) 
        ttk.Label(r2frame, font=self.TFont1, foreground='red',
                   textvariable=self.qd1_txt).grid(column=3, row=3, sticky=(W)) 
        ttk.Label(r2frame, font=self.TFont1, foreground='green',
                   textvariable=self.qd2r_txt).grid(column=3, row=4, sticky=(W)) 
        ttk.Label(r2frame, font=self.TFont1, foreground='red',
                   textvariable=self.qd2_txt).grid(column=3, row=5, sticky=(W)) 
        #  joint acceleration -- ref. v.s. actual
        ttk.Label(r2frame, font=self.TFont1, foreground='darkgreen',
                   textvariable=self.qdd1r_txt).grid(column=4, row=2, sticky=(W)) 
        ttk.Label(r2frame, font=self.TFont1, foreground='red',
                   textvariable=self.qdd1_txt).grid(column=4, row=3, sticky=(W)) 
        ttk.Label(r2frame, font=self.TFont1, foreground='darkgreen',
                   textvariable=self.qdd2r_txt).grid(column=4, row=4, sticky=(W)) 
        ttk.Label(r2frame, font=self.TFont1, foreground='red',
                   textvariable=self.qdd2_txt).grid(column=4, row=5, sticky=(W)) 
        
        # --- torque display frame -----
        tauframe = ttk.Frame(mainframe, padding = (2,2,2,2),relief="sunken")
        tauframe.grid(column=1,row=cvrowspan+1)
        ttk.Label(tauframe, text="Torque          ", 
                  font=self.TFont1,foreground='maroon').grid(column=0, row=0, sticky=(N,W))        
        ttk.Label(tauframe, text="J1 : ",font=self.TFont1,
                  foreground='darkblue').grid(column=0, row=1, sticky=W)
        ttk.Label(tauframe, text="J2 : ",font=self.TFont1,
                 foreground='darkblue').grid(column=0, row=2, sticky=W)
        ttk.Label(tauframe, font=self.TFont1, foreground='olive',
                   textvariable=self.tau1_txt).grid(column=1, row=1, sticky=(W)) 
        ttk.Label(tauframe, font=self.TFont1, foreground='olive',
                   textvariable=self.tau2_txt).grid(column=1, row=2, sticky=(W)) 
        
       
        #   ------ feedback group.  ------------------------------
        fbframe = ttk.Frame(mainframe, padding = (2,2,2,2),relief="sunken")
        fbframe.grid(column=2, row=0) #column=cvcolspan,row=0)        
        # self.feedback_txt.set("1")  
        ttk.Label(fbframe, text="Feedback          ", 
                  font=self.TFont1,foreground='maroon').grid(column=0, row=0, sticky=(N,W))        
        self.olrb = (ttk.Radiobutton(fbframe,text="Open-loop", variable=self.feedback_txt,
                                     value="0",command=self.send_cmd_fbopen).
                                     grid(column=0, row=1, sticky=W))
        self.clrb = (ttk.Radiobutton(fbframe, text="Closed-loop", variable=self.feedback_txt,
                                     value="1",command=self.send_cmd_fbclosed).
                                     grid(column=0, row=2, sticky=W))

        # -----  controller group. Changed ----------------------------------------------
        ctlframe = ttk.Frame(mainframe, padding = (2,2,2,2),relief="sunken")
        ctlframe.grid(column=2,row=1)        
        # self.controller_txt.set("0")
        ttk.Label(ctlframe, text="Controller        ", font=self.TFont1,foreground='magenta').grid(column=0, row=0, sticky=(N,W))        
        self.pidrb = (ttk.Radiobutton(ctlframe,text="PDG", variable=self.controller_txt,
                                      value="0",command=self.send_cmd_pdg_control).grid(column=0, 
                                                                            row=1, sticky=W))
        self.ccrb = (ttk.Radiobutton(ctlframe, text="Inverse Dynamic", variable=self.controller_txt,
                                     value="1",command=self.send_cmd_invdyn_control).grid(column=0,
                                                                            row=2, sticky=W))
        self.sfbrb = (ttk.Radiobutton(ctlframe, text="Adaptive", variable=self.controller_txt,
                                     value="2",command=self.send_cmd_adaptive_control).grid(column=0, 
                                                                            row=3, sticky=W))

        # -------- choice of inverse kinematics ----------------------
        # ikine1s, ikine2s  (step )
        # ikine1t, ikine2t  (track)
        # ---------------------------------------------------------------------------
        ikframe = ttk.Frame(mainframe, padding = (2,2,2,2),relief="sunken")
        ikframe.grid(column=2,row=2) #,rowspan=7)
        ttk.Label(ikframe, text="Inverse kinematics", 
                  font=self.TFont1,foreground='green').grid(column=0, row=0, sticky=W)
        ttk.Label(ikframe, text="   Step", 
                  font=self.TFont1,foreground='crimson').grid(column=0, row=1, sticky=W)
        self.iktype_txt.set("0")
        self.ik1srb = (ttk.Radiobutton(ikframe,text="ikine1s", variable=self.iktype_txt,
                                      value="0").grid(column=0, row=2, sticky=W))
        self.ik2srb = (ttk.Radiobutton(ikframe, text="ikine2s", variable=self.iktype_txt,
                                     value="1").grid(column=0, row=3, sticky=W))
        ttk.Label(ikframe, text="   Track", 
                  font=self.TFont1,foreground='royalblue').grid(column=0, row=4, sticky=W)

        
        self.ik1trb = (ttk.Radiobutton(ikframe, text="ikine1t", variable=self.iktype_txt,
                                     value="2").grid(column=0, row=5, sticky=W))
        self.ik2trb = (ttk.Radiobutton(ikframe, text="ikine2t", variable=self.iktype_txt,
                                     value="3").grid(column=0, row=6, sticky=W))
        
        #  -------- cartesian coordinate frame ------------------
        
        xyframe = ttk.Frame(mainframe, padding = (2,2,2,2),relief="sunken")
        xyframe.grid(column=2,row=3)
        ttk.Label(xyframe, text="XY coordinates", 
                  font=self.TFont1,foreground='green').grid(column=0, row=0, sticky=W)
        ttk.Label(xyframe, text="X goal: ", 
                  font=self.TFont1,foreground='blue').grid(column=0, row=1, sticky=W)
        ttk.Label(xyframe, textvariable=self.xg_txt, 
                  font=self.TFont1,foreground='teal').grid(column=1, row=1, sticky=W)
        ttk.Label(xyframe, text="Y goal: ", 
                  font=self.TFont1,foreground='blue').grid(column=0, row=2, sticky=W)
        ttk.Label(xyframe, textvariable=self.yg_txt, 
                  font=self.TFont1,foreground='teal').grid(column=1, row=2, sticky=W)        
        ttk.Label(xyframe, textvariable=self.ikmsg_txt, 
                  font=self.TFont1,foreground='orangered').grid(column=0, row=3, sticky=W)
        
        ttk.Label(xyframe, text="X tool: ", 
                  font=self.TFont1,foreground='blue').grid(column=0, row=4, sticky=W)
        ttk.Label(xyframe, textvariable=self.xt_txt, 
                  font=self.TFont1,foreground='teal').grid(column=1, row=4, sticky=W)
        ttk.Label(xyframe, text="Y tool: ", 
                  font=self.TFont1,foreground='blue').grid(column=0, row=5, sticky=W)
        ttk.Label(xyframe, textvariable=self.yt_txt, 
                  font=self.TFont1,foreground='teal').grid(column=1, row=5, sticky=W)
        ttk.Button(xyframe, text="Move", command=self.send_goal).grid(column=0, 
                                                        row=6, sticky=W)

        # ------- draw path control ------
        self.drawmode = BooleanVar()
        self.showws = BooleanVar()
        self.showhist = BooleanVar()
        drawframe = ttk.Frame(mainframe, padding = (2,2,2,2),relief="sunken")
        drawframe.grid(column=2,row=4)
        ttk.Label(drawframe, text="Plot control         ", 
                  font=self.TFont1,foreground='olive').grid(column=0, row=0, sticky=W)
        ttk.Checkbutton(drawframe, text="draw path",variable=self.drawmode).grid(column=0, 
                                                                row=1, sticky=W)
        ttk.Button(drawframe, text="Clear", command=self.clear_path).grid(column=0, 
                                     row=2, sticky=W)        
        ttk.Checkbutton(drawframe, text="show workspace",variable=self.showws).grid(column=0, 
                                    row=3, sticky=W)
        ttk.Checkbutton(drawframe, text="show history",variable=self.showhist).grid(column=0, 
                                    row=4, sticky=W)

#         self.draw_btn = ttk.Button(drawframe, text="Clear", command=self.toggle_draw).grid(column=0, 
#                                     row=1, sticky=W) # toggle between "Draw" and "Clear"
        
        ttk.Label(mainframe, text="dew.ninja 2023", font=self.TFont1, 
                  foreground="darkred").grid(column=2,row=5,sticky=W)
        
        # ------ canvas for 2-link manipulator plot  ----------
        self.x_vec = []  # used for path draw
        self.y_vec = []
        self.fig, self.ax = plt.subplots(figsize=(9,9))
        cif = self.fig.canvas.mpl_connect('button_press_event',self.mouse_event) 

        self.point_of_rotation = np.array([0, self.link_height/2])         
        # -----------------------------------------
        
#         self.time_step = 0.0
#         self.time_vec = []
#         self.y1_vec = []
#         self.y2_vec = []
#         self.y3_vec = []
       
#         self.fig, self.axes  = plt.subplots(2,1)
#         self.ax1, self.ax2 = self.fig.get_axes()
        self.canvas = FigureCanvasTkAgg(self.fig, master=mainframe)
        self.canvas.get_tk_widget().grid(row=0,column=0, rowspan=cvrowspan, columnspan=cvcolspan, sticky=W)
        self.an1 = FuncAnimation(self.fig, self.animate, interval=1000,cache_frame_data=False )
#      save_count=MAX_FRAMES or cache_frame_data=False
        

        for child in mainframe.winfo_children(): 
            child.grid_configure(padx=5, pady=5)
        
        self.Client_ID = "" # supply your NETPIE info 
        self.User = ""
        self.Password = ""    
        self.broker = "broker.netpie.io"
        self.port = 1883     
        if self.online:
            self.mqtt_connect()

        self.t = 0.05
        self.plantsim = 0
        self.datasize = 200
        self.cmdgen_flag = 0
        self.feedback = 1
        self.controller = 0
        self.issolution = False # flag whether ikine solution exists or not
        self.iktype = 0  # type of inverse kinematics 
        # 2-link robot parameters for kinematics
        self.l1 = 1.0
        self.l2 = 1.0
        
        
        # initialize randomly to aid in GUI design 
        self.q1r = np.random.rand()
        self.qd1r = np.random.rand()
        self.qdd1r = np.random.rand()
        self.q2r = np.random.rand()
        self.qd2r = np.random.rand()
        self.qdd2r = np.random.rand()
        self.q1 = np.random.rand()
        self.qd1 = np.random.rand()
        self.qdd1 = np.random.rand()
        self.q2 = np.random.rand()
        self.qd2 = np.random.rand()
        self.qdd2 = np.random.rand()
        self.tau1 = np.random.rand()
        self.tau2 = np.random.rand()
        
        self.xg = 0.0  # goal coordinate. Start at (0.0,2.0)
        self.yg = 2.0
        self.xt = 0.0   # tool coordinate
        self.yt = 0.0
        self.q1g = 0.0  # target angle, in degrees
        self.q2g = 0.0 
        
        self.q1r_deg = self.rad2deg*self.q1r
        self.q1_deg = self.rad2deg*self.q1
        self.q2r_deg = self.rad2deg*self.q2r
        self.q2_deg = self.rad2deg*self.q2
        

        self.q1r_txt.set(str(round(self.q1r,3)))
        self.qd1r_txt.set(str(round(self.qd1r,3)))
        self.qdd1r_txt.set(str(round(self.qdd1r,3)))
        self.q2r_txt.set(str(round(self.q2r,3)))
        self.qd2r_txt.set(str(round(self.qd2r,3)))
        self.qdd2r_txt.set(str(round(self.qdd2r,3)))
        self.q1_txt.set(str(round(self.q1,3)))
        self.qd1_txt.set(str(round(self.qd1,3)))
        self.qdd1_txt.set(str(round(self.qdd1,3)))
        self.q2_txt.set(str(round(self.q2,3)))
        self.qd2_txt.set(str(round(self.qd2,3)))
        self.qdd2_txt.set(str(round(self.qdd2,3)))
        self.tau1_txt.set(str(round(self.tau1,3)))
        self.tau2_txt.set(str(round(self.tau2,3)))

        self.q1r_deg_txt.set(str(round(self.q1r_deg,3)))
        self.q1_deg_txt.set(str(round(self.q1_deg,3)))
        self.q2r_deg_txt.set(str(round(self.q2r_deg,3)))
        self.q2_deg_txt.set(str(round(self.q2_deg,3)))
        
        self.xg_txt.set(str(round(self.xg,2)))
        self.yg_txt.set(str(round(self.yg,2)))
        

    def mqtt_connect(self):
        print("Connecting to mqtt broker")
        try:
            self.client = mqtt.Client(client_id=self.Client_ID,
                         transport='tcp',
                         protocol=mqtt.MQTTv311,
                         clean_session=True)
            self.client.username_pw_set(self.User,self.Password)
            self.client.connect(self.broker, port=self.port, keepalive=60)
            self.client.subscribe("@msg/update")
            #self.client.subscribe("@msg/shadowdata")
            self.client.on_message = self.on_message
            self.client.loop_start()
        except:
            print("Problem connecting to mqtt broker")

    def on_message(self,client, userdata, message):
        self.rcvd_msg = str(message.payload.decode("utf-8"))
        self.rcvd_topic = message.topic
        #print("message received ", self.rcvd_msg)
        #print("message topic=",message.topic)

        if self.rcvd_topic == "@msg/update":
            parm_data = self.rcvd_msg.split(',')
            # strings for GUI update
            self.t_txt.set(parm_data[0])
            self.plantsim_txt.set(parm_data[1]) 
            self.datasize_txt.set(parm_data[2])
            self.cmdgen_txt.set(parm_data[3])  # change to cmdgen_flag
            self.feedback_txt.set(parm_data[4]) 
            self.controller_txt.set(parm_data[5]) 
            self.q1r_txt.set(parm_data[6])
            self.qd1r_txt.set(parm_data[7])
            self.qdd1r_txt.set(parm_data[8])
            self.q2r_txt.set(parm_data[9])
            self.qd2r_txt.set(parm_data[10])
            self.qdd2r_txt.set(parm_data[11])
            self.q1_txt.set(parm_data[12])
            self.qd1_txt.set(parm_data[13])
            self.qdd1_txt.set(parm_data[14])
            self.q2_txt.set(parm_data[15])
            self.qd2_txt.set(parm_data[16])
            self.qdd2_txt.set(parm_data[17])
            self.tau1_txt.set(parm_data[18])
            self.tau2_txt.set(parm_data[19])
            
            # update parameters
            self.t = float(parm_data[0])
            self.plantsim = int(parm_data[1]) 
            self.datasize = int(parm_data[2]) 
            self.capture = int(parm_data[3]) 
            self.feedback = int(parm_data[4]) 
            self.controller = int(parm_data[5]) 
            self.q1r = float(parm_data[6])
            self.qd1r = float(parm_data[7])
            self.qdd1r = float(parm_data[8])
            self.q2r = float(parm_data[9])
            self.qd2r = float(parm_data[10])
            self.qdd2r = float(parm_data[11])
            self.q1 = float(parm_data[12])
            self.qd1 = float(parm_data[13])
            self.qdd1 = float(parm_data[14])
            self.q2 = float(parm_data[15])
            self.qd2 = float(parm_data[16])
            self.qdd2 = float(parm_data[17])
            self.tau1 = float(parm_data[18])
            self.tau2 = float(parm_data[19])
            
            # convert to degree for display purpose
            self.q1r_deg = self.rad2deg*self.q1r
            self.q1_deg = self.rad2deg*self.q1
            self.q2r_deg = self.rad2deg*self.q2r
            self.q2_deg = self.rad2deg*self.q2
            
            self.q1r_deg_txt.set(str(round(self.q1r_deg,2)))            
            self.q1_deg_txt.set(str(round(self.q1_deg,2)))            
            self.q2r_deg_txt.set(str(round(self.q2r_deg,2)))            
            self.q2_deg_txt.set(str(round(self.q2r_deg,2)))            
            
    def animate(self,i):
        if not self.showhist.get():  # delete previous plot or not
            self.ax.cla()
        self.ax.set_aspect('equal')
        self.ax.set_xlim(-2.1,2.1)
        self.ax.set_ylim(-2.1,2.1)
    
        self.ax.plot(0, 0,  color='orange', marker='o', markersize=30)
        self.ax.plot(0, 0,  color='black', marker='o',fillstyle="none", markersize=30)
        self.ax.plot(0, 0,  color='gray', marker='o', markersize=10)
        
        #self.q1 = 3*np.random.rand()  # test
        
        self.q1_deg = self.rad2deg*self.q1
        # ---- test ---------------
#         self.q1_deg = self.q1g  # test
#         self.q1 = self.q1_deg/self.rad2deg # test
        #-------------------------------------
        
        link1 = RotatingRectangle((0, 0), width=self.link_width, height=self.link_height, 
                            rel_point_of_rot=self.point_of_rotation,
                            angle=self.q1_deg, edgecolor='black', facecolor='gray') # color='0.5', alpha=0.9,)
        self.ax.add_patch(link1)
        # find the center of link 2
        
        x1 = np.cos(self.q1)
        y1 = np.sin(self.q1)        
        self.ax.plot(x1, y1,  color='orange', marker='o', markersize=30)
        self.ax.plot(x1, y1,  color='black', marker='o',fillstyle="none", markersize=30)        
        self.ax.plot(x1,y1,  color='gray', marker='o', markersize=10)
        
#        print(self.q2)

        #self.q2 = 3*np.random.rand()  # test
    
        self.q2_deg = self.rad2deg*self.q2
        
        # ------- test ----------------
#         self.q2_deg = self.q2g  # test 
#         self.q2 = self.q2_deg/self.rad2deg
        #-----------------------------------
        link2 = RotatingRectangle((x1,y1), width=self.link_width, height=self.link_height, 
                            rel_point_of_rot=self.point_of_rotation,
                            angle=self.q1_deg+self.q2_deg, edgecolor='black', facecolor='gray')
        self.ax.add_patch(link2)
        # find tool coordinates
        # q2 = np.pi*q2deg/180
        x2 = np.cos(self.q1+self.q2)
        y2 = np.sin(self.q1+self.q2)
        self.xt = x1+x2
        self.yt = y1+y2
        self.ax.plot(self.xt, self.yt,  color='g', marker='o', markersize=10)
        self.ax.grid(True)   
        self.ax.set_xlabel("x")
        self.ax.set_ylabel("y")
        
        self.xt_txt.set(str(round(self.xt,2)))
        self.yt_txt.set(str(round(self.yt,2)))

        self.ax.plot(self.xg, self.yg,  color='r', marker='X', markersize=15) # target position

        if self.showws.get(): # draw workspace
            robo_ws = plt.Circle((0.0,0.0),2, color='m',fill=False)
            self.ax.add_patch(robo_ws)
            
        if self.drawmode.get():   # draw path
            self.x_vec.append(self.xt)
            self.y_vec.append(self.yt)
            self.ax.plot(self.x_vec,self.y_vec,'blue')
#        print("Animating")

            
#     def dbupdate(self, *args):
#         self.client.publish("@msg/cmd","dbupdate")

    def send_r2(self, *args):  # changed to send arbitrary command
        cmd_txt = self.r2_txt.get()
        if self.online:
            self.client.publish("@msg/cmd",cmd_txt)
        
    def send_cmd_fbopen(self, *args):
        if self.online:
            cmd_txt = "feedback=0"
            self.client.publish("@msg/cmd",cmd_txt)
    def send_cmd_fbclosed(self, *args):
        if self.online:
            cmd_txt = "feedback=1"
            self.client.publish("@msg/cmd",cmd_txt)
    def send_cmd_pdg_control(self, *args):  # changed
        if self.online:
            cmd_txt = "controller=0"
            self.client.publish("@msg/cmd",cmd_txt)
    def send_cmd_invdyn_control(self, *args): # changed
        if self.online:        
            cmd_txt = "controller=1"
            self.client.publish("@msg/cmd",cmd_txt)
    def send_cmd_adaptive_control(self, *args): # changed
        if self.online:        
            cmd_txt = "controller=2"
            self.client.publish("@msg/cmd",cmd_txt)

    def send_goal(self, *args): # send step or track command to robot controller
        if self.online:
            self.iktype = self.iktype_txt.get() # ikine type from radio button
            if self.iktype == '0' or self.iktype == '1': # ikine1s or ikine2s
                cmd_txt = "r="+str(round(self.q1g,3))+","+str(round(self.q2g,3))
                self.client.publish("@msg/cmd",cmd_txt)
            else: # ikine1t or ikine2t
                cmd_txt = "track="+str(round(self.q1g,3))+","+str(round(self.q2g,3))
                self.client.publish("@msg/cmd",cmd_txt)

#     def check_draw(self, *args):  # toggle path draw
#         print("Drawmode = ",self.drawmode.get())
    def clear_path(self, *args):         
        self.x_vec = []
        self.y_vec = []
#     def show_workspace(self, *args):         
#         pass

    def mouse_event(self,event):
        self.xg = round(event.xdata,3)
        self.yg = round(event.ydata,5)
        self.xg_txt.set(str(self.xg))
        self.yg_txt.set(str(self.yg))
        self.ikine()
        
    # compute inverse kinematics depending on  
    # value of self.iktype_txt.get()
    def ikine(self, *args): 
        # first check if solution exist
        c2 = (self.xg**2 + self.yg**2 - self.l1**2 - self.l2**2)/(2*self.l1*self.l2)
        if abs(c2)>1.0: # no solution
            self.issolution = False
            self.ikmsg_txt.set("No solution")
        else:
            self.issolution = True
            self.iktype = self.iktype_txt.get() # ikine type from radio button
            if self.iktype == '0' or self.iktype == '2': # ikine1s or ikine1t
                s2 = np.sqrt(1 - c2**2)
            else: # ikine2s or ikine2t
                s2 = -np.sqrt(1-c2**2)
            q2g = np.arctan2(s2,c2)
            k1 = self.l1 + self.l2*c2
            k2 = self.l2*s2
            q1g = np.arctan2(self.yg,self.xg)-np.arctan2(k2,k1)
            self.q1g = self.rad2deg*q1g
            self.q2g = self.rad2deg*q2g
            self.ikmsg_txt.set("q1,q2: ("+str(round(self.q1g,3))+","+str(round(self.q2g,3))+")")
            # print(self.ikmsg_txt.get())
        
#     def send_cmd_kp(self, *args):
#         cmd_txt = "kp="+self.kp_txt.get()
#         self.client.publish("@msg/cmd",cmd_txt)
        
    def on_close(self):
        print("Cleaning up.. ")
        if self.online:
            self.client.loop_stop()
            self.client.disconnect()
        root.destroy()

root = Tk()
r2Controller(root)
root.mainloop()
    

Connecting to mqtt broker
